# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operacional`, de `Arenales, Armentano, Morabito e Yanasse`.</sup>

## Exercício 1

### Descrição do problema
Uma fundição tem de produzir 10 toneladas de um tipo de liga metálica e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração, em termos percentuais, desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínima e máxima de cada componente da liga).

Frações dos elementos (%) nos ingredientes e custo dos ingredientes (R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.5 | 90 | 9 |
| Silício | 14 | - | 27 |
| Custo | 90 | 180 | 25 |

Frações (%) mínima e máxima dos componentes na liga:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 9.5 |
|Silício | 19 | 20 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

In [1]:
# instalação e importação do pacote mip
!pip install mip

from mip import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 29.3 MB/s eta 0:00:00


Carrega Dados

In [ ]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14},
    'g': {'c': 0.9,   's': 0.0},
    's': {'c': 0.09,  's': 0.27},
}

# custo
c = {'l': 90, 'g': 180, 's': 25}

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min
m = {'c': 0.095, 's': 0.2} # max

# quantidade desejada da liga
Q = 10

Cria modelo

In [35]:
model = Model(sense=MINIMIZE, solver_name=CBC)

# Criação da variável x
x = [model.add_var(var_type=CONTINUOUS, name=f"x_{i}", lb=0.0) for i in range(3)]

# Função objetivo = minimizar custo de produção
model.objective = 0.09*x[0] + 0.18*x[1] + 0.025*x[2]

# Adição das restrições para a quantidade de carbono
model += 0.005*x[0] + 0.9*x[1] + 0.09*x[2] >= 0
model += 0.005*x[0] + 0.9*x[1] + 0.09*x[2] <= 950

# Adição das restrições para a quantidade de silício
model += 0.14*x[0] + 0.27*x[2] >= 1900
model += 0.14*x[0] + 0.27*x[2] <= 2000

# Restrição de garantia de produção total
model += x[0] + x[1] + x[2] == 10000

model.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 0.09000 x_0 + 0.18000 x_1 + 0.02500 x_2
Subject To
constr(0):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(1):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 950
constr(2):  0.14000 x_0 + 0.27000 x_2 >= 1900
constr(3):  0.14000 x_0 + 0.27000 x_2 <= 2000
constr(4):  x_0 + x_1 + x_2 = 10000
Bounds
End



Executa

In [36]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")
  
  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
x_0 = 5384.62
x_1 = 0.00
x_2 = 4615.38


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Código

Carrega Dados

In [ ]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

In [47]:
model = Model(sense=MINIMIZE, solver_name=CBC)
x = [model.add_var(var_type=CONTINUOUS, name=f"x_{i}", lb=0.0) for i in range(3)]

# Função objetivo = minimizar custo de produção
model.objective = 0.09*x[0] + 0.18*x[1] + 0.025*x[2]

# Adição das restrições para a quantidade de carbono
model += 0.005*x[0] + 0.9*x[1] + 0.09*x[2] >= 0
model += 0.005*x[0] + 0.9*x[1] + 0.09*x[2] <= 950

# Adição das restrições para a quantidade de silício
model += 0.14*x[0] + 0.27*x[2] >= 1900
model += 0.14*x[0] + 0.27*x[2] <= 2000
model += x[0] <= 5000
model += x[1] <= 5000
model += x[2] <= 12000

# Restrição de garantia de produção total
model += x[0] + x[1] + x[2] == 10000

model.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 0.09000 x_0 + 0.18000 x_1 + 0.02500 x_2
Subject To
constr(0):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(1):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 950
constr(2):  0.14000 x_0 + 0.27000 x_2 >= 1900
constr(3):  0.14000 x_0 + 0.27000 x_2 <= 2000
constr(4):  x_0 <= 5000
constr(5):  x_1 <= 5000
constr(6):  x_2 <= 12000
constr(7):  x_0 + x_1 + x_2 = 10000
Bounds
End



Executa

In [48]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
x_0 = 5000.00
x_1 = 185.19
x_2 = 4814.81


## Exercício 3

Suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e sua composição mínima e máxima é dada na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 0.4 |
|Silício | 0.12 | 0.19 |


### Código

Carrega dados

In [ ]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, {'c': 0.0, 's': 0.12}]
m = [{'c': 0.095, 's': 0.2}, {'c': 0.4, 's': 0.19}]

# quantidade desejada da liga
Q = [10, 6]

Cria modelo

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

# Função objetivo = minimizar custo de produção
model.objective = 0.09*x[0] + 0.18*x[1] + 0.025*x[2]

# Adição das restrições para a quantidade de carbono
model += 0.005*x[0] + 0.9*x[1] + 0.09*x[2] >= 0
model += 0.005*x[0] + 0.9*x[1] + 0.09*x[2] <= 950

# Adição das restrições para a quantidade de silício
model += 0.14*x[0] + 0.27*x[2] >= 1900
model += 0.14*x[0] + 0.27*x[2] <= 2000
model += x[0] + x[1] + x[2] == 6000



model.write("modelo3.lp") # salva modelo em arquivo
with open("modelo3.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

Executa

In [ ]:
solve(model)